In [9]:
%%time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import cross_validation, ensemble, tree, preprocessing, neighbors, naive_bayes
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
le = preprocessing.LabelEncoder()
import xgboost as xgb
from datetime import datetime, date
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using Theano backend.


CPU times: user 295 ms, sys: 711 ms, total: 1.01 s
Wall time: 3.9 s


In [3]:
%%time
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 8.82 µs


In [ ]:
%%time
ids = []
predictions = []

In [ ]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
# %%time
test = pd.read_csv('test.csv', dtype=test_dtypes)
train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=100000)
# train = pd.concat([chunk[chunk['user_location_city'].isin(np.unique(test.user_location_city))] for chunk in train], ignore_index=True)
# train = pd.concat([chunk[chunk['is_booking'] == 1] for chunk in train], ignore_index=True)
train = pd.concat([chunk for chunk in train], ignore_index=True)
# train = pd.concat([chunk[(chunk['user_id'].isin(np.unique(test.user_id))) & (chunk['is_booking'] == 1)] for chunk in train], ignore_index=True)
# train = train[train['user_id'].isin(np.unique(test.user_id))]
destinations = pd.read_csv('destinations.csv')

In [ ]:
print len(train), len(test)

In [ ]:
%%time
train['id'] = [i for i in range(0, len(train))]
train = train.drop(['cnt'], 1)
test['is_booking'] = 1
train['is_test'] = 0
test['is_test'] = 1
test['hotel_cluster'] = 0

In [ ]:
%%time
train = pd.concat([train, test], ignore_index=True)

In [ ]:
%%time
destination_ids = destinations['srch_destination_id']
destinations = destinations.drop(['srch_destination_id'], 1)
pca = PCA(n_components=3, whiten=True)
destinations = pca.fit_transform(destinations)
destinations = pd.DataFrame(destinations)
destinations['srch_destination_id'] = destination_ids

In [ ]:
%%time
train['orig_destination_distance'] = train['orig_destination_distance'].fillna(-1)
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
train['srch_ci'] = pd.to_datetime(train['srch_ci'], errors='coerce')
train['srch_co'] = pd.to_datetime(train['srch_co'], errors='coerce')
train['activity_month'] = train['date_time'].fillna(-1).dt.month.astype(int)
train['activity_year'] = train['date_time'].fillna(-1).dt.year.astype(int)
train['activity_dow'] = train['date_time'].fillna(-1).dt.dayofweek.astype(int)
train['activity_day'] = train['date_time'].fillna(-1).dt.day.astype(int)
train['activity_quarter'] = train['date_time'].fillna(-1).dt.quarter.astype(int)
train['checkin_month'] = train['srch_ci'].fillna(-1).dt.month.astype(int)
train['checkin_year'] = train['srch_ci'].fillna(-1).dt.year.astype(int)
train['checkin_dow'] = train['srch_ci'].fillna(-1).dt.dayofweek.astype(int)
train['checkin_day'] = train['srch_ci'].fillna(-1).dt.day.astype(int)
train['checkin_quarter'] = train['srch_ci'].fillna(-1).dt.quarter.astype(int)
train['checkout_month'] = train['srch_co'].fillna(-1).dt.month.astype(int)
train['checkout_year'] = train['srch_co'].fillna(-1).dt.year.astype(int)
train['checkout_dow'] = train['srch_co'].fillna(-1).dt.dayofweek.astype(int)
train['checkout_day'] = train['srch_co'].fillna(-1).dt.day.astype(int)
train['checkout_quarter'] = train['srch_co'].fillna(-1).dt.quarter.astype(int)
train['stay_length'] = (train['srch_co'] - train['srch_ci']).astype(int)

In [ ]:
print len(train)

In [ ]:
# %%time
test['orig_destination_distance'] = test['orig_destination_distance'].fillna(-1)
test['date_time'] = pd.to_datetime(test['date_time'], errors='coerce')
test['srch_ci'] = pd.to_datetime(test['srch_ci'], errors='coerce')
test['srch_co'] = pd.to_datetime(test['srch_co'], errors='coerce')
test['activity_month'] = test['date_time'].fillna(-1).dt.month.astype(int)
test['activity_year'] = test['date_time'].fillna(-1).dt.year.astype(int)
test['activity_dow'] = test['date_time'].fillna(-1).dt.dayofweek.astype(int)
test['activity_day'] = test['date_time'].fillna(-1).dt.day.astype(int)
test['activity_quarter'] = test['date_time'].fillna(-1).dt.quarter.astype(int)
test['checkin_month'] = test['srch_ci'].fillna(-1).dt.month.astype(int)
test['checkin_year'] = test['srch_ci'].fillna(-1).dt.year.astype(int)
test['checkin_dow'] = test['srch_ci'].fillna(-1).dt.dayofweek.astype(int)
test['checkin_day'] = test['srch_ci'].fillna(-1).dt.day.astype(int)
test['checkin_quarter'] = test['srch_ci'].fillna(-1).dt.quarter.astype(int)
test['checkout_month'] = test['srch_co'].fillna(-1).dt.month.astype(int)
test['checkout_year'] = test['srch_co'].fillna(-1).dt.year.astype(int)
test['checkout_dow'] = test['srch_co'].fillna(-1).dt.dayofweek.astype(int)
test['checkout_day'] = test['srch_co'].fillna(-1).dt.day.astype(int)
test['checkout_quarter'] = test['srch_co'].fillna(-1).dt.quarter.astype(int)
test['stay_length'] = (test['srch_co'] - test['srch_ci']).astype(int)

In [ ]:
# %%time
train = pd.merge(train, destinations, how='left')
train.fillna(-1, inplace=True)

In [ ]:
# %%time
test = pd.merge(test, destinations, how='left')
test.fillna(-1, inplace=True)

In [ ]:
%%time
train.to_csv('combined_data.csv', index=False)

In [4]:
%%time
train = pd.read_csv('combined_data.csv', iterator=True, chunksize=100000)
# train = pd.concat([chunk[chunk['user_location_city'].isin(np.unique(test.user_location_city))] for chunk in train], ignore_index=True)
# train = pd.concat([chunk[chunk['is_booking'] == 1] for chunk in train], ignore_index=True)
train = pd.concat([chunk for chunk in train], ignore_index=True)

CPU times: user 4min 58s, sys: 2min 48s, total: 7min 46s
Wall time: 9min 40s


In [5]:
%%time
train.dtypes

CPU times: user 705 µs, sys: 2.5 ms, total: 3.21 ms
Wall time: 9.29 ms


channel                        int64
date_time                     object
hotel_cluster                  int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
id                             int64
is_booking                     int64
is_mobile                      int64
is_package                     int64
is_test                        int64
orig_destination_distance    float64
posa_continent                 int64
site_name                      int64
srch_adults_cnt                int64
srch_children_cnt              int64
srch_ci                       object
srch_co                       object
srch_destination_id            int64
srch_destination_type_id       int64
srch_rm_cnt                    int64
user_id                        int64
user_location_city             int64
user_location_country          int64
user_location_region           int64
activity_month                 int64
activity_year                  int64
a

In [7]:
%%time
features = [c for c in train.columns if c not in ['id', 'hotel_cluster', 'date_time', 'srch_ci', 'srch_co', 'is_booking', 'is_test']]

CPU times: user 53 µs, sys: 19 µs, total: 72 µs
Wall time: 67 µs


In [8]:
%%time
print len(train[features][train['is_test'] == 0])

37670293


In [12]:
%%time
encoded = [c for c in train.columns if c not in ['id', 'hotel_cluster', 'date_time', 'srch_ci', 'srch_co', 'is_booking', 'is_test', 'stay_length', '0', '1', '2', 'orig_destination_distance']]

CPU times: user 85 µs, sys: 126 µs, total: 211 µs
Wall time: 204 µs


In [13]:
%%time
train_encoded = np_utils.to_categorical(train[encoded])

TypeError: cannot convert the series to <type 'int'>

In [ ]:
train_encoded.shape

In [ ]:
print len(np.unique(train.user_id)), len(np.unique(test.user_id))

In [ ]:
# %%time
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(train[features], train['hotel_cluster'], test_size=0.50)
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features_train, labels_train, test_size=0.60)

In [ ]:
# %%time
# neigh = neighbors.KNeighborsClassifier(weights='distance', n_jobs=-1).fit(train[features], train['hotel_cluster'])
forest = ensemble.RandomForestClassifier(n_estimators=10, n_jobs=-1).fit(train[features], train['hotel_cluster'])
# bayes = naive_bayes.GaussianNB().fit(train[features], train['hotel_cluster'])

In [ ]:
# %%time
probs = pd.DataFrame(forest.predict_proba(test[features]))
probs.columns = np.unique(train['hotel_cluster'].sort_values().values)
# probs.columns = np.unique(labels_train.sort_values().values)
# probs.columns = np.unique(labels_train.values)
preds = pd.DataFrame([list([r.sort_values(ascending=False)[:5].index.values]) for i,r in probs.iterrows()])
# print mapk([[l] for l in labels_test], preds[0], 5)

In [ ]:
# %%time
submission = pd.DataFrame()
submission['id'] = test['id']
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in preds.values]
submission.sort_values(by='id', inplace=True)

In [ ]:
# %%time
submission.head()

In [ ]:
# %%time
submission.to_csv('submission.csv', index=False)